In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras
from BinanceCollector import BinanceCollector
from TradeBook import TradeBook
import warnings
warnings.filterwarnings("ignore")
from BreakdownCalculator import BreakdownCalculator
from BinanceSync import BinanceSync
from HyperBacktest import HyperBacktest
import matplotlib.pyplot as plt
from datetime import datetime,timedelta

c:\Users\deves\AppData\Local\Programs\Python\Python37\lib\site-packages\requests\__init__.py:114: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  RequestsDependencyWarning,


In [2]:
start = (datetime.now() - timedelta(days=60)).strftime("%Y-%m-%d")
end = (datetime.now()).strftime("%Y-%m-%d")
symbol = "ETHUSDT"
interval = "15m"

In [3]:
collector = BinanceCollector(symbol,start,end,interval)
collector.start_collect()
collector.df.columns = ["Open Time","Open","High","Low","Close","Volume","Close Time",
             "Quote asset volumne","Total Trades","Buyer Base Volume",
             "Taker buy quote asset volume","Ignore"]
df = collector.df
df["Open Time"] = pd.to_datetime(df["Open Time"],unit="ms")
df["Close Time"] = pd.to_datetime(df["Close Time"],unit="ms")

In [6]:
done = False
while True:
    # Change according to interval
    if (datetime.now().minute-1)%15 != 0:
        done = False
        continue
    if done:
        continue
    print("Startng sync...")
    sync = BinanceSync(df,symbol,interval)
    sync.sync()
    df = sync.df
    df["EMA_20"] = df["Close"].ewm(span=20,adjust=False).mean()
    df["EMA_50"] = df["Close"].ewm(span=50,adjust=False).mean()
    df["EMA_200"] = df["Close"].ewm(span=200,adjust=False).mean()
    df["Buy"] = np.where( (df["EMA_20"] > df["EMA_50"]) & (df["EMA_20"].shift(1) <= df["EMA_50"].shift(1) ),1,0 )
    df["Sell"] = np.where( (df["EMA_20"] < df["EMA_50"]) & (df["EMA_20"].shift(1) >= df["EMA_50"].shift(1) ),1,0 )
    print(df.iloc[-1])
    print("Synced!")
    if df.iloc[-1]["Buy"] == 1:
        print(f"Buy Signal at {df.iloc[-1]['Close Time']}")
    if df.iloc[-1]["Sell"] == 1:
        print(f"Sell Signal at {df.iloc[-1]['Close Time']}")
    done = True

Startng sync...
Stopping
Start: 1672941599999.0
 Stop: 1672941660191.0
Last candle: 2023-01-05 17:59:59.999000
Current Time: 2023-01-05 23:31:00.191000
Dataframe End: 2023-01-05 17:44:59.999000
Open Time                              2023-01-05 17:45:00
Open                                               1246.98
High                                               1247.38
Low                                                1246.36
Close                                              1246.79
Volume                                             17316.5
Close Time                      2023-01-05 17:59:59.999000
Quote asset volumne                                    NaN
Total Trades                                           NaN
Buyer Base Volume                                      NaN
Taker buy quote asset volume                           NaN
Ignore                                                   0
Quote Asset Volume                              2.1591e+07
Number of Trades                       

In [4]:
sync = BinanceSync(df,symbol,interval)
sync.sync()
sync.df.tail()

Stopping
Last candle: 2023-01-05 17:59:59.999000
Current Time: 2023-01-05 23:22:03.305000


,Open Time,Open,High,Low,Close,Volume,Close Time,Quote asset volumne,Total Trades,Buyer Base Volume,Taker buy quote asset volume,Ignore,Quote Asset Volume,Number of Trades,Taker Buy Base Asset Volume,Taker Buy Quote Asset Volume
5766,2023-01-05 16:30:00,1247.67,1248.80,1246.93,1248.50,20526.008,2023-01-05 16:44:59.999,NaN,NaN,NaN,NaN,0.0,2.560858e+07,9506.0,13151.371,1.640794e+07
5767,2023-01-05 16:45:00,1248.50,1248.77,1247.63,1248.22,10291.964,2023-01-05 16:59:59.999,NaN,NaN,NaN,NaN,0.0,1.284589e+07,6430.0,4922.448,6.144038e+06
5768,2023-01-05 17:00:00,1248.21,1249.74,1247.74,1248.62,17249.921,2023-01-05 17:14:59.999,NaN,NaN,NaN,NaN,0.0,2.154145e+07,8151.0,9943.550,1.241745e+07
5769,2023-01-05 17:15:00,1248.62,1249.00,1246.97,1247.88,14903.985,2023-01-05 17:29:59.999,NaN,NaN,NaN,NaN,0.0,1.859962e+07,7309.0,7409.841,9.247522e+06
5770,2023-01-05 17:30:00,1247.87,1248.90,1246.77,1246.98,16212.646,2023-01-05 17:44:59.999,NaN,NaN,NaN,NaN,0.0,2.023090e+07,7536.0,6129.870,7.649385e+06


In [5]:
df = sync.df

In [5]:
1672940397137 - 1672940699999

-302862